In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [6]:
video_path = 'hw2_2/ftfm.mp4'
output_folder = 'output/'

if os.path.isdir(output_folder):
    print("Delete old result folder: {}".format(output_folder))
    os.system("rm -rf {}".format(output_folder))
os.system("mkdir {}".format(output_folder))
print("create folder: {}".format(output_folder))

vc = cv2.VideoCapture(video_path)
fps = vc.get(cv2.CAP_PROP_FPS)
frame_count = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
print(frame_count)
video = []


create folder: output/
770


In [7]:
t = frame_count
digit = 0
while t > 0:
    t = int(t / 10)
    digit += 1
print(digit)

3


In [8]:
hist_base = []
prev_hist = []
frame_flag = np.zeros(frame_count)
for idx in range(0, 1380, 1):
    vc.set(1, idx)
    ret, frame = vc.read()
    height, width, layers = frame.shape
    size = (width, height)
    #print(frame)
    #img = cv2.imread('hw2_1\\news_out\\news-0000000.jpg')
    h_bins = 100
    s_bins = 100
    histSize = [h_bins, s_bins]
    # hue varies from 0 to 179, saturation from 0 to 255
    h_ranges = [0, 180]
    s_ranges = [0, 256]
    ranges = h_ranges + s_ranges # concat lists
    # Use the 0-th and 1-st channels
    channels = [0, 1]
    

    if frame is not None:
        #cv2.imshow("preview", frame)
        #cv2.waitKey(0)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #hist = cv2.calcHist([gray], [0], None, [256], [0, 256])

        t = []
        for i in range(4):
            for j in range(4):
                h = int(size[1]/4)
                w = int(size[0]/4)
                t.append(cv2.calcHist([gray[h*i:h*(i+1)-1, w*j:w*(j+1)-1]], [0], None, [256], [0, 256]))
                cv2.normalize(t[i*4+j], t[i*4+j], alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
        hist_base.append(t)

        # 畫出直方圖
        #print(hist)
        #print(hist_base)
        #print(prev_hist)
        
        #hist = hist.flatten()
        #plt.bar(range(1,257), hist)
        #plt.show()
        #plt.bar(range(1,257), hist_base)
        #plt.show()
        if idx != 0:
            for compare_method in range(1):
                base_base = []
                gap = 15
                for i in range(16):
                    t = cv2.compareHist(hist_base[idx][i], hist_base[idx - 1][i], compare_method)
                    base_base.append(t)
                #for i in range(1, 4):
                    #if idx - i*gap >= 0:
                        #t = cv2.compareHist(hist_base[idx], hist_base[idx - i*gap], compare_method)
                        #base_base.append(t) 
                #for i in range(4):
                    #if len(prev_hist[0]) != 0:
                        #t = cv2.compareHist(hist_base, prev_hist[0], i)
                        #base_base.append(t) 
                #if base_base < 1:
                if len(base_base) > 0:
                    #print(idx, ' Method:', compare_method, 'Perfect:',\
                    #'[{:.4f}, {:.4f}, {:.4f}, {:.4f}]'.format(base_base[0], base_base[1], base_base[2], base_base[3]), '/')
                    s = ''
                    for i in range(len(base_base)):
                        if base_base[i] < 0.75:
                            
                            s += ' {:.4f}'.format(base_base[i])
                        else:
                            s += '       '
                    
                    print('{:4}'.format(idx), ' Method:', compare_method, ' ',\
                    s, '/')
    else:
        print('error')
        
s = ''
for i in range(len(frame_flag)):
    if frame_flag[i]:
        print(i)

   1  Method: 0    -0.0069 -0.0104 -0.0134 0.0291 -0.0159 -0.0159 -0.0105 -0.0087 -0.0093 -0.0097 -0.0094 -0.0058 -0.0116 -0.0132 -0.0066 0.1314 /
   2  Method: 0    -0.0175 -0.0433 -0.0111 0.1912 0.1767 0.3000 0.1137 0.1021 -0.0241 -0.0113 0.0977 0.2684 0.1220 0.1467 0.2397        /
   3  Method: 0    -0.0229 -0.0895 -0.0207 0.2930 0.2627 0.2133 0.2627 0.4365 0.1301 0.1962 0.4608 0.6737 0.4190 0.5080 0.7068 0.5254 /
   4  Method: 0    -0.0218 -0.0037 0.2075 0.4610 0.4702 0.1546 0.4621 0.6975 0.4364 0.4467 0.6790        0.6327 0.6806               /
   5  Method: 0    -0.0234 0.2019 0.3956 0.6015 0.5948 0.1868 0.6870        0.6677 0.6745                                           /
   6  Method: 0    -0.0127 0.4416 0.5922 0.7357 0.6917 0.2656                                                                       /
   7  Method: 0    0.0231 0.7282                      0.5218                                                                       /
   8  Method: 0    0.3423                  

AttributeError: 'NoneType' object has no attribute 'shape'